In [ ]:
import jax
import jax.numpy as jnp
from typing import NamedTuple, Callable, Optional
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from functools import cache

In [ ]:
DIRECTIONS = jnp.array([(1, 0), (0, 1), (-1, 0), (0, -1)])
ND = len(DIRECTIONS)
RIGHT, UP, LEFT, DOWN = range(ND)  # in clockwise order

Position = jnp.ndarray  # (x, y)
TileType = int  # index into the tileset
DirectionSet = jnp.ndarray

# Constructing the Tileset

In [ ]:
class Tileset(NamedTuple):
    tiles: pd.DataFrame
    neighbors: pd.DataFrame
    subsets: pd.DataFrame

def load_tileset(path: str) -> Tileset:
    groups = ['tiles', 'neighbors', 'subsets']
    tiles, neighbors, subsets = [pd.read_xml(path, xpath=f'./{group}/*') for group in groups]
    subsets = {name: pd.read_xml(path, xpath=f'./subsets/subset[@name="{name}"]/*') for name in subsets['name']}
    return Tileset(tiles, neighbors, subsets)

In [ ]:
circuit_tiles = load_tileset('./tilesets/Circuit.xml')
circuit_tiles.tiles

## Tile rotations

I'll use "tile" to refer to the original, unrotated tiles, and "TILE" to refer to an element of the massive set of rotated tiles we obtain afterwards.

$C$ denotes the number of TILEs.

In [ ]:
class Symmetry(NamedTuple):
    card: int
    """The number of elements in the symmetry group."""

    rotate: Callable[[int], int]
    """A function that rotates an element of the symmetry group."""

    flip: Callable[[int], int]
    """A function that flips an element of the symmetry group."""

id = lambda i: i
rotate = lambda i: (i + 1) % 4                     # rotates clockwise                         (1, 2, 3, 0)
flipa = lambda i: i ^ 1                            # flips across the diagonal y=x             (2, 3, 0, 1)
flipb = lambda i: jnp.where(i % 2 == 0, i, 4 - i)  # flips across the x-axis                   (3, 0, 1, 2)
flipc = lambda i: 1 - i                            # same as flipa but using negative indices  (1, 0, -1, -2)

SYMMETRIES = {
    'X': Symmetry(1, id, id),
    'L': Symmetry(4, rotate, flipa),
    'T': Symmetry(4, rotate, flipb),
    'I': Symmetry(2, flipc, id),
    '\\': Symmetry(2, flipc, flipc),
    'F': Symmetry(8, lambda i: jnp.where(i < 4, rotate(i), 4 + rotate(i)), lambda i: i ^ 4),
}

def _get_symmetry(sym: Symmetry, t=0):
    s = []
    for _ in range(4):
        s.append((t, sym.flip(t)))
        t = sym.rotate(t)
    return jnp.array(s)

@cache
def get_symmetries(symmetry: str) -> list[Symmetry]:
    s = SYMMETRIES[symmetry]
    return jax.vmap(_get_symmetry, in_axes=(None, 0))(s, jnp.arange(s.card))

In [ ]:
def show_symmetries(tile: str, symmetry: str):
    fig, axs = plt.subplots(2, 5, figsize=(10, 4))
    img = plt.imread(f'./tilesets/Circuit/{tile}.png')
    axs[0, 0].imshow(img)
    axs[0, 0].axis('off')
    axs[0, 0].set_title(f"{tile} ({symmetry})")

    axs[1, 0].axis('off')

    for j, k in enumerate(_get_symmetry(SYMMETRIES[symmetry])):
        i = j + 1
        k0, k1 = k[0].item(), k[1].item()
        axs[0, i].imshow(jnp.rot90(img, k0))
        axs[1, i].imshow(jnp.rot90(img, k1))
        axs[0, i].axis('off')
        axs[1, i].axis('off')
        axs[0, i].set_title(k0)
        axs[1, i].set_title(k1)
    plt.show()

In [ ]:
show_symmetries('substrate', 'X')

In [ ]:
show_symmetries('skew', 'L')

In [ ]:
show_symmetries('bridge', 'I')

In [ ]:
show_symmetries('connection', 'T')

In [ ]:
show_symmetries('dskew', '\\')

In [ ]:
def load_patterns(tiles: Tileset):
    """
    Turns the set of tiles into the set of TILEs.
    """
    names = []
    spins = []
    stepsto = []
    weights = []

    for name, symmetry, weight in tiles.tiles.itertuples(index=False):
        s = get_symmetries(symmetry)  # Shape (sym.card, 4, 2)
        card = s.shape[0]
        names.extend([name] * card)
        spins.extend(range(card))
        stepsto.extend(s + len(stepsto))
        weights.extend([weight] * card)

    index = pd.MultiIndex.from_arrays([names, spins], names=['name', 'spin'])
    return pd.DataFrame({
        'stepsto': stepsto,
        'weight': weights,
    }, index=index)

In [ ]:
patterns = load_patterns(circuit_tiles)
patterns

# Propagation

Great! Now that we have the set of rotations set up,
we need to be able to tell which tile types are allowed to be
next to each other. We'll do this with a `propagator` matrix defined below.

In [ ]:
def _preprocess(left, right):
    """Turn a name in the 'neighbors' table into a (name, spin) pair for indexing into the df."""
    left, right = left.split(' '), right.split(' ')
    if len(left) == 1:
        left += ['0']
    if len(right) == 1:
        right += ['0']
    return (left[0], int(left[1])), (right[0], int(right[1]))

def create_propagator(tileset: Tileset, stepsto: pd.DataFrame) -> jnp.ndarray:
    """Shape (D, C, C). propagator[dir, from, to] is True if
    the tile `from` is compatible with tile `to` in direction `dir`.
    For example, propagator[RIGHT, L, R] means that the tile pattern L -> R is allowed."""

    # use regular numpy temporarily since it's more convenient
    propagator = np.empty((ND, len(stepsto), len(stepsto)), dtype=np.bool_)

    for pair in tqdm(tileset.neighbors.itertuples(index=False), total=len(tileset.neighbors)):
        l, r = _preprocess(*pair)
        lstep, rstep = stepsto.loc[[l, r], 'stepsto']
        dstep = stepsto.iloc[lstep[1, 0].item()]['stepsto']
        ustep = stepsto.iloc[rstep[1, 0].item()]['stepsto']

        propagator[LEFT, rstep[0, 0], lstep[0, 0]] = True
        propagator[LEFT, lstep[0, 1], rstep[0, 1]] = True
        propagator[LEFT, lstep[2, 0], rstep[2, 0]] = True
        propagator[LEFT, rstep[2, 1], lstep[2, 1]] = True

        propagator[UP, ustep[0, 0], dstep[0, 0]] = True
        propagator[UP, dstep[0, 1], ustep[0, 1]] = True
        propagator[UP, ustep[2, 1], dstep[2, 1]] = True
        propagator[UP, dstep[2, 0], ustep[2, 0]] = True
    
    propagator[RIGHT] = propagator[LEFT].T
    propagator[DOWN] = propagator[UP].T

    return jnp.array(propagator)

In [ ]:
propagator = create_propagator(circuit_tiles, patterns)
propagator.shape

# State

Now let's build the actual algorithm for "collapsing the wave function"!
The "wave function" itself is a boolean array. For each (node, tile type) pair,
it tracks whether or not this pair is still valid.

In [ ]:
class State(NamedTuple):
    """The state of the WFC algorithm."""

    wave: jnp.ndarray
    """Shape (W, H, C). wave[x, y, k] is True if pattern k is possible at
    position (x, y), and False otherwise."""

    compatible: jnp.ndarray
    """Shape (W, H, D, C). compatible[x, y, d, k] counts how many ways there are to get
    type k at position (x, y) _from_ direction d."""

    entropies: jnp.ndarray
    """Shape (W, H). The entropy of each position. Used for picking the next state."""

    def ntypes(self):
        return self.wave.shape[-1]
    
    def shape(self):
        return self.wave.shape[:2]
    
    def positions(self):
        w, h = self.shape()
        return jnp.mgrid[:w, :h].reshape(2, -1).T
    
    def counts(self):
        return self.wave.sum(axis=-1)
    
    def is_collapsed(self):
        """Shape (W, H). True if the wave is collapsed at the given position."""
        return self.counts() == 1
    
class Problem(NamedTuple):
    """The problem to be solved by the WFC algorithm."""

    weights: jnp.ndarray
    """The marginal unnormalized positive probability assigned to each pattern."""

    propagator: jnp.ndarray
    """Shape (D, C, C). propagator[dir, from, to] is True if the tile `from` is
    compatible with tile `to` in direction `dir`."""

class Ban(NamedTuple):
    """A ban on a particular pattern at a particular position."""

    node: Position
    banned: jnp.ndarray
    dirs: Optional[DirectionSet] = ...

In [ ]:
def init_state(problem: Problem, W: int, H: int) -> State:
    """Initializes the state of the WFC algorithm."""
    C = problem.weights.shape[0]
    wave = jnp.ones((W, H, C), dtype=jnp.bool_)
    entropies = jnp.full((W, H), entropy(problem.weights))
    # TODO make sure to generalize this for arbitrary directions
    counts = problem.propagator.sum(axis=-1)  # (D, C) how many tiles does each tile connect to (in that direction)?
    compatible = jnp.broadcast_to(counts[jnp.arange(ND) ^ 2], (W, H, ND, C))
    return State(wave, compatible, entropies)

@jax.jit
def entropy(weights: jnp.ndarray):
    """Weights should be the unnormalized positive probability of each tile type.
    Not to be confused with logits = log(weights)."""
    s = jnp.sum(weights, axis=-1)
    w = jnp.where(weights > 0, weights * jnp.log(weights), 0)  # avoid log zero errors
    return jnp.log(s) - jnp.sum(w, axis=-1) / s

In [ ]:
problem = Problem(patterns['weight'].values, propagator)
state = init_state(problem, 24, 24)

In [ ]:
def run(key, state: State, problem: Problem, max_steps=10_000):
    """Run Wave Function Collapse for a maximum of `max_steps` steps."""
    for _ in tqdm(range(max_steps)):
        key, subkey = jax.random.split(key)
        node = next_node(subkey, state)

        key, subkey = jax.random.split(key)
        state, banned = collapse(subkey, state, problem.weights, node)
        state, ok = propagate(state, problem, Ban(node, banned))
        if not ok:
            return state, False
        
        if state.is_collapsed().all():
            return state, True

    return state, True

@jax.jit
def next_node(key, state: State, std=1e-4) -> Optional[Position]:
    """
    Get a minimum-entropy state that has not yet been fully observed.
    If all states are fully observed, return None.
    Assumes that the wave is not fully collapsed.
    """
    ka, kb = jax.random.split(key)
    entropies = jnp.where(state.is_collapsed(), jnp.inf, state.entropies)  # don't pick collapsed positions
    noise = jax.random.normal(ka, entropies.shape) * std
    p = jnp.isclose(entropies + noise, entropies.min())  # a mask (W, H) of positions with minimum entropy
    p = jnp.where(p, 1, 0).reshape(-1) / p.sum()  # choose uniformly from those positions
    return jax.random.choice(kb, state.positions(), p=p)

@jax.jit
def collapse(key, state: State, weights: jnp.array, node: Position) -> tuple[State, jnp.ndarray]:
    """'Collapse' the wave at the given node by sampling a given pattern
    according to the weights. Eliminate all other patterns at that node."""
    x, y = node
    p = jnp.where(state.wave[x, y, :], weights, 0)
    ary = jnp.arange(state.ntypes())
    pattern = jax.random.choice(key, ary, p=p / p.sum())
    banned = (ary != pattern) & state.wave[x, y, :]  # the types that have been eliminated
    state = eliminate(state, weights, Ban(node, banned))
    return state, banned

@jax.jit
def eliminate(state: State, weights: jnp.ndarray, ban: Ban) -> State:
    """Eliminate the given tile types at the given position. banned_types has shape (C,)."""
    x, y = ban.node
    banned = (~ban.banned).astype(state.compatible.dtype)
    wave = state.wave.at[x, y].multiply(banned)  # eliminate the banned tile_types at (x, y)
    # eliminate the banned tile_types from the compatible matrix in all directions
    compatible = state.compatible.at[x, y, ban.dirs].multiply(banned)
    h = entropy(jnp.where(wave[x, y], weights, 0))  # the entropy of the remaining types
    entropies = state.entropies.at[x, y].set(h)
    return State(wave, compatible, entropies)

@jax.jit
def _propagate(state: State, problem: Problem, ban: Ban) -> tuple[State, Ban]:
    """
    Propagate the constraints from the given position (2, U) and the given banned types (C,).
    directions should be a list of directions to propagate in, e.g. (UP, RIGHT).
    """
    x, y = (ban.node + DIRECTIONS[ban.dirs]).T
    # e.g. if pos = (5, 5) "T" just got banned,
    # then for d = RIGHT,
    # we need to eliminate "T" from the left-compatible types at (6, 5).
    eliminated = jnp.where(ban.banned, problem.propagator[ban.dirs].transpose(0, 2, 1), 0).sum(axis=-1)  # (D, C). Flip to enable broadcasting.
    eliminated = eliminated.astype(state.compatible.dtype)
    compatible = state.compatible.at[x, y, ban.dirs].add(-eliminated)  # update the compatible counts
    banned = (eliminated > 0) & (compatible[x, y, ban.dirs] == 0)  # the types that have just been eliminated
    state = state._replace(compatible=compatible)
    state = eliminate(state, problem.weights, Ban(jnp.array([x, y]), banned, ban.dirs))
    return state, Ban(jnp.array([x, y]), banned)


def propagate(state: State, problem: Problem, ban: Ban) -> tuple[State, bool]:
    """Each element of the stack is a (2 + C) array indicating the position and the list of banned types at that position.
    The True elements have been eliminated and the changes need to be propagated."""

    stack: list[Ban] = [ban]
    canvas = jnp.array(state.shape())

    while stack:  # the stack contains position-type pairs that have just been eliminated
        ban = stack.pop()
        p = ban.node + DIRECTIONS
        dirs, = jnp.nonzero(((p >= 0) & (p < canvas)).all(axis=-1))
        state, ((xx, yy), ban, _) = _propagate(state, problem, ban._replace(dirs=dirs))
        for x, y, z in zip(xx, yy, ban):
            if z.any():
                stack.append(Ban(jnp.array((x, y)), z))

    return state, True


In [ ]:
with jax.disable_jit():
    result = run(jax.random.PRNGKey(0), state, problem)